In [2]:
select top 10 * from INFORMATION_SCHEMA.COLUMNS

(10 rows affected)

Total execution time: 00:00:00.009

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,NUMERIC_PRECISION,NUMERIC_PRECISION_RADIX,NUMERIC_SCALE,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
Denver Crime,dbo,Land_Transfer,PARID,1,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,RECEPTION_NUM,2,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,INSTRUMENT,3,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_YEAR,4,NULL,YES,float,NULL,NULL,53,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_MONTHDAY,5,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,RECEPTION_DATE,6,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_PRICE,7,NULL,YES,float,NULL,NULL,53,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,GRANTOR,8,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,GRANTEE,9,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,CLASS,10,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL


In [3]:
-- Aggregate Crime dataset for the studied period (2010 - 2013)
WITH Crime_Data 
AS 
(
SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2010
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')

UNION ALL

SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2011
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')

UNION ALL

SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2012
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')
)

SELECT OFFENSE_CATEGORY, Year, Month, SUM(Count) as Count, NBHD_Name_Crime
INTO Crime_Data
FROM Crime_Data
WHERE REPORTED_DATE < '2013-01-01'
GROUP BY OFFENSE_CATEGORY, Year, Month, NBHD_Name_Crime
ORDER BY Year, Month

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'Crime_Data' in the database.

Total execution time: 00:00:00.016

In [6]:
SELECT top 10 *
FROM Crime_Data


(10 rows affected)

Total execution time: 00:00:00.001

OFFENSE_CATEGORY,Year,Month,Count,NBHD_Name_Crime
theft from motor vehicle,2010,9,2,regis
public disorder,2011,11,14,montbello
auto theft,2012,3,8,highland
auto theft,2011,4,3,mar lee
drug alcohol,2011,5,2,north capitol hill
white collar crime,2011,4,1,hampden south
burglary,2011,11,5,lincoln park
theft from motor vehicle,2011,10,2,skyland
white collar crime,2012,8,3,north capitol hill
drug alcohol,2010,5,2,gateway green valley ranch


In [4]:
-- Aggregate crime neighborhood names across the studied period and join them to land trasfer dataset on neighborhood name
SELECT distinct top 10 L.NBHD_1 as NBHD_No, L.NBHD_1_CN AS NBHD_Name_Sales, C.NBHD_Name_Crime
FROM Crime_Data C
Full Join Land_Transfer L on L.NBHD_1_CN = C.NBHD_Name_Crime
order by NBHD_No

(10 rows affected)

Total execution time: 00:00:01.289

NBHD_No,NBHD_Name_Sales,NBHD_Name_Crime
NULL,NULL,NULL
NULL,NULL,barnum west
NULL,NULL,city park
NULL,NULL,city park west
NULL,NULL,college view south platte
NULL,NULL,dia
NULL,NULL,east colfax
NULL,NULL,elyria swansea
NULL,NULL,gateway green valley ranch
NULL,NULL,hale


In [5]:
-- the output is inspected in Excel to match the remaining neighborhood names to produce Neighborhood Mapping Key


-- Join Real Estate Sales data on Crime data using the Neighborhood Mapping Key on Neighborhood ID & Name and on month and year
SELECT top 10 *
FROM (SELECT ROW_NUMBER() OVER (ORDER BY NBHD) AS SN, * FROM SalesBook) S
Left join Neighborhood_Mapping M on S.NBHD = CAST(M.NBHD_No AS float)
Inner join Crime_Data C on M.NBHD_Name_Crime = C.NBHD_Name_Crime
	and S.SMONTH = C.Month
	and S.SYEAR = C.Year
ORDER by SN

(10 rows affected)

Total execution time: 00:00:00.041

SN,Model,Cluster,Group,CondoComplex,CondoComplexName,NBHD,PARCEL,ABSTRPRD,LIVEAREA,FINBSMNT,BASEMENT,YRBUILT,CONDITION,QUALITY,TOTUNITS,STORIES,GARTYPE,NOCARS,XTRAFFIC,FLOORLVL,NUMBDRM,NUMBATHS,COMMUSE,ARCSTYLE,SPRICE,DEDUCT,NETPRICE,TASP,SMONTH,SYEAR,QUALIFIED,STATUS,NBHD_No,NBHD_Name_Sales,NBHD_Name_Crime,OFFENSE_CATEGORY,Year,Month,Count,NBHD_Name_Crime
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,auto theft,2011,9,5,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,sexual assault,2011,9,3,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,other crimes against persons,2011,9,9,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,aggravated assault,2011,9,2,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,all other crimes,2011,9,4,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,drug alcohol,2011,9,5,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,larceny,2011,9,17,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,theft from motor vehicle,2011,9,9,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,robbery,2011,9,6,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,burglary,2011,9,29,montbello
